In [ ]:
# download network and weights
!wget -N https://raw.githubusercontent.com/grip-unina/GANimageDetection/main/resnet50nodown.py
!wget -N --no-check-certificate -e robots=off -nd -P ./weights -A .pth -r http://www.grip.unina.it/download/GANdetection

In [ ]:
import os
import glob
import time
from PIL  import Image
from datetime import datetime
import random

date = datetime.now().strftime("%Y-%m-%d_%H%M%S")

In [ ]:
glide_config = ['inpainting', 'text2img']
config = glide_config[1]

fake_folder = f"/content/drive/MyDrive/datasets/glide/{config}"
real_folder = "/content/drive/MyDrive/datasets/coco/test2017"

weights_path = './weights/gandetection_resnet50nodown_stylegan2.pth'

out_folder = "/content/drive/MyDrive/signal_prj/glide"

In [ ]:
from resnet50nodown import resnet50nodown

from torch.cuda import is_available as is_available_cuda
device = 'cuda:0' if is_available_cuda() else 'cpu'

net = resnet50nodown(device, weights_path)

In [ ]:
fake = sum([glob.glob(os.path.join(fake_folder, '*.png'))], list())
real = sum([glob.glob(os.path.join(real_folder, '*.jpg'))], list())[-1000:]

imgs = fake + real
random.shuffle(imgs)
imgs_len = len(imgs)

In [ ]:
def run():
  output_csv = f'{out_folder}/out.{date}.{config}.csv'    
  
  with open(output_csv,'w') as fid:
      fid.write('filename,time,fake,logit\n')
      fid.flush()

      for index, filename in enumerate(imgs):          
          isFake = "glide" in filename

          tstart = time.time()
          img = Image.open(filename).convert('RGB')
          img.load()
          logit = net.apply(img)
          tend = time.time()
        
          outstr = '%s,%f,%i,%f\n' % (filename, tend-tstart, isFake, logit)
          print(outstr)
          fid.write(outstr)
          fid.flush()

In [ ]:
run()